In [1]:
import geopandas as gpd
import pandas as pd
from shapely import wkt
from concurrent.futures import ProcessPoolExecutor
from pathlib import Path
import os

/home/sandeep/miniconda3/envs/geoai/lib/python3.11/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_3076666/669706969.py:1: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are usi

In [18]:
from geoai.db.postgres import get_engine

In [2]:
def parse_file(filepath, chunk_size=1000000):
    low = 0
    while True:
        skiprows = low
        high = low + chunk_size
        out_path = filepath.parent.parent / 'feather' / (filepath.name.split('.')[0] + f'_{low}_{high}.feather')
        low+=chunk_size
        if out_path.exists():
            continue
        
        print(skiprows)
        df = pd.read_csv(filepath, skiprows=range(1, skiprows), nrows=chunk_size)
        if len(df) == 0:
            break
        df['geometry'] = df['geometry'].apply(wkt.loads)
        df = gpd.GeoDataFrame(df, geometry='geometry', crs=4326)
        df.to_postgis()
    print(f"Processed: {filepath}")

In [3]:
files = list(Path('/home/sandeep/workspace/data/google-building-footprints/v3/csv').glob('*.csv.gz'))
print(f"-- Processing total files: {len(files)}")
num_workers = 1
# for i in range(0, len(files), num_workers):
#     print(f"-- Progress: {i}/{len(files)}")
# pool = ProcessPoolExecutor(max_workers=num_workers)
# res = pool.map(parse_file, files)
# pool.shutdown()

for i, file in enumerate(files):
    print(f"-- Progress: {i}/{len(files)} | Processing File: {file}")
    parse_file(file)

-- Processing total files: 22
-- Progress: 0/22 | Processing File: /home/sandeep/workspace/data/google-building-footprints/v3/csv/393_buildings.csv.gz
36000000
Processed: /home/sandeep/workspace/data/google-building-footprints/v3/csv/393_buildings.csv.gz
-- Progress: 1/22 | Processing File: /home/sandeep/workspace/data/google-building-footprints/v3/csv/38f_buildings.csv.gz
5000000
Processed: /home/sandeep/workspace/data/google-building-footprints/v3/csv/38f_buildings.csv.gz
-- Progress: 2/22 | Processing File: /home/sandeep/workspace/data/google-building-footprints/v3/csv/3b1_buildings.csv.gz
17000000
Processed: /home/sandeep/workspace/data/google-building-footprints/v3/csv/3b1_buildings.csv.gz
-- Progress: 3/22 | Processing File: /home/sandeep/workspace/data/google-building-footprints/v3/csv/3af_buildings.csv.gz
11000000
Processed: /home/sandeep/workspace/data/google-building-footprints/v3/csv/3af_buildings.csv.gz
-- Progress: 4/22 | Processing File: /home/sandeep/workspace/data/googl

In [4]:
from geoai.db.postgres import start_container

In [5]:
start_container()

In [6]:
from sqlalchemy import create_engine

In [8]:
from geoai import config

In [15]:
engine = create_engine(f'postgresql://postgres:@localhost:54320/{config.POSTGRES_DB_NAME}', pool_size=20, max_overflow=0)

In [16]:
con = engine.connect()